In [1]:
#import preprocessed per user dataset from onedrive prickled file
import pickle
import os
import numpy as np

path = r"/Users/nathantc5/OneDrive - The University of Sydney (Students)/CS48-CAPSTONE Project 2021 Sem1/dataset"
#create output dictionary
data = {}

N_user = 1000

for u in os.listdir(path+'/user_7_days')[:N_user]: #import 100users

    uid = int(u[:-7])
    file_name = '/'.join([path,'/user_7_days', u])

    try:
        with open(file_name, 'rb') as f:
            dic = pickle.load(f)
            data[uid] = dic[uid]
    except:
        print(dic)
total_user = len(data.keys())
print("Finish importing {} users".format(total_user))


Finish importing 870 users


In [2]:
import pandas as pd
weather = pd.read_csv('/'.join([path,'/weather_data.txt']), delimiter="\t")
weather_np = weather.T.to_numpy()
weather_dic = {20:weather_np[0], 21:weather_np[1], 22: weather_np[2], 23:weather_np[2], 24:weather_np[2],25:weather_np[3], 26:weather_np[4]}

In [9]:
weather_dic[20][1:]

(47,)

In [2]:
# #create 2 function to extract the feature and label from array
# def extract_time(lst):
#     #extract first columns (session) from ROWS
#     return [item[:1] for item in lst]

# def extrac_place(lst):
#     #extract second columns (mode base_station) from ROWS
#     return [item[1] for item in lst]

# def extract_app(lst):
#     #extract app_usage columns from ROWS
#     return np.array([item[2:] for item in lst])

# # function to generate input for baseline model
# def generate_baseline_input(data):

#     data_neural = data

#     all_prev_app = []
#     all_target_app = []
#     all_prev_t = []
#     all_target_t = []

#     day_id = [26] #using all the days for training

#     # if candidate is None:
#     candidate = data_neural.keys() #filter, and get user id

#     #iterate all the users
#     for u in candidate:
#         #seperate feature and label list
#         #get user's record
#         sessions = data_neural[u]
#         #sepearate to store pred and true
#         user_prev_app = []
#         user_target_app = []
#         user_prev_t = []
#         user_target_t = []

#         for i in day_id:
#             #call specific day
#             session = sessions[i] #= data[u][i]
#             # session is (48,2002)
#             # idea here is to iterate over all 48 rows to extract the first columns and last 2000 columns

#             #extract app_usage
#             app_usage = np.array(extract_app(session))
#             #extract time
#             time_usage = np.array(extract_time(session))
#             weather_day = weather_dic[i] #(48,)
    


#             #slice the app section to get app usage of ind 0 to second last rows (47,2000)
#             prev_app = np.array(app_usage)[:-1,]
#             #slice the app section to get app usage of ind 1 to last rows (47,2000)
#             target_app = np.array(app_usage)[1:,]

#             prev_t = np.array(time_usage)[:-1,]
#             target_t = np.array(time_usage)[1:,]
#             weather_day = weather_day[1:,] #(47,)

#             user_prev_app.append(prev_app)
#             user_target_app.append(target_app)
#             user_prev_t.append(prev_t)
#             user_target_t.append(target_t)
#             # print(current_session[24])
#             # print(previous_session[24])
#         #after iterate the day, need to shift the feature columns upwards for prediction
#         #also need to cut the last rows for labels
#         #reshape: as orginal dataset were seperate by day,session,column:
#         #feature:4(days),48sessions,2features
#         #label:4(days),48sessions,2000 app usage(count)
#         #reshape to flattern the data
#         # user_prev = np.array(user_prev).reshape(-1,2000)
#         # user_true = np.array(user_true).reshape(-1,2000)
#         # print(user_prev[24])
#         # print(user_true[24])
#         #append per user's feature and label to all_user list
#         all_prev_app.append(user_prev_app)
#         all_target_app.append(user_target_app)
#         all_prev_t.append(user_prev_t)
#         all_target_t.append(user_target_t)

#     all_prev_app= np.array(all_prev_app)
#     all_target_app = np.array(all_target_app)
#     all_prev_t = np.array(all_prev_t)
#     all_target_t = np.array(all_target_t)

#     return all_prev_app, all_target_app, all_prev_t, all_target_t

In [40]:
#create 2 function to extract the feature and label from array
#just to keep every clean as possible

def extract_timeloc(lst):
    #extract first 2 columns: session and mode location 
    return [item[:2] for item in lst]


def extract_label(lst):
    #extract the remaning columns
    return [item[2:2002] for item in lst]


def extract_poi(lst):
    return [item[2002:] for item in lst]

# function to generate input for baseline model
# return(no.of users, rows of a users, feature dimension)
def generate_baseline_input(data, mode):
    data_neural = data
    data_feature = []
    data_label = []

    if mode == 'train':
        day_id = [20, 21, 22] #the day for training

    elif mode == 'test':
        day_id = [26] # the day for testing

    # if candidate is None:
    candidate = data_neural.keys() #filter, and get user id

    #iterate all the users
    for u in candidate:
        #seperate feature and label list
        user_X_train = []
        user_y_label = []
        #get user's record
        sessions = data_neural[u]
        #seperate days for training and testing
        for i in day_id:
            #call specific day
            session = data_neural[u][i]
            #extract the part we want
            # print(session.shape)

            timeloc = np.array(extract_timeloc(session)[1:])
            prev_app_count = np.array(extract_label(session)[:-1])
            target = np.array(extract_label(session)[1:])
            poi = np.array(extract_poi(session)[1:]) 

            weather_day = weather_dic[i]
            weather_day = weather_day[1:]
            weather_day = weather_day.reshape(-1,1)# weather_day from (47,) to (47,1)

            # print(timeloc.shape)
            # print(prev_app_count.shape)
            # print(weather_day.shape)

            #concate feature
            user_feature = np.concatenate((timeloc,prev_app_count,weather_day,poi),axis = 1)
            # print(user_feature.shape)
            #append feature and label to corresponding list
            user_X_train.append(user_feature)
            user_y_label.append(target)
            # print(user_feature.shape)

        #reason for reshape here: group 1 user at 1 dimension when append to all_user list
        user_X_train = np.array(user_X_train).reshape(-1,2020) #2 columns: timeloc, 2000 columns, app-usage, 17 columnns: POI, 1 columns:weather = 2020
        user_y_label = np.array(user_y_label).reshape(-1,2000)
        #append user_data to all_use list
        data_feature.append(user_X_train)
        data_label.append(user_y_label)
    # no reshape here because when calling the variables it is easy to check how many user have been imported\
    # therefore it returns 3 dimenionsal data : (no.of users, rows of a users, feature dimension)
    data_feature = np.array(data_feature)
    data_label = np.array(data_label)

    return data_feature, data_label


In [41]:
X_train, y_train = generate_baseline_input(data, 'train')
X_test, y_test = generate_baseline_input(data,'test')
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)
# print("X_train has {} user, each user has {} rows, each row has {} columns". format(X_train.shape[0], X_train.shape[1], X_train.shape[2]))

(870, 141, 2020)
(870, 141, 2000)
(870, 47, 2020)
(870, 47, 2000)


# Helper function

In [5]:
#metrics function

from sklearn import metrics as skmetrics

def cal_ap( y_actual, y_pred, k ):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # Calculation of precision
    num = 0
    rank = 0
    sumP = 0.0
    for item in s_zip_topk:
        rank += 1
        if item[0] == 1:
            num += 1
            sumP += (num*1.0)/(rank*1.0)
    ap = 0.0
    if num > 0:
        ap = sumP/(num*1.0)
    return ap   # average precision
# Take topk prediction and the ground truth

def r_k(y_actual, y_pred, k, threshold):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # print(s_zip_topk)
    actual, pred = zip(*s_zip_topk)
    actual = np.where(np.array(actual) > threshold, 1, 0)
    pred = np.array(pred)
    pred_o = np.where(pred > threshold, 1, 0)
    return actual, pred_o, pred


def user_evaluation_metrics(y_pred, y_test):
    total_auc = 0
    total_map = 0
    total_recall = 0
    v_count = 0
    count = 0

    y_pred = np.where(y_pred >0,1,0)
    y_test = np.where(y_test >0,1,0)
    # print(y_pred.shape)
    # print(y_test.shape)

    for i in range(y_test.shape[0]):

        if (np.sum(y_test[i])> 0):
            fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], y_pred[i], pos_label=1)
            total_auc += skmetrics.auc(fpr, tpr)

            actual, pred_o, pred = r_k(y_test[i], y_pred[i] ,5, 0.5)
            total_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5

            total_map += cal_ap(y_test[i], y_pred[i], 2000)
            #divide only the valid rows that has data
            v_count +=1

        else:
            count +=1
            pass
            
    if v_count != 0:
        total_auc = total_auc / v_count
        total_map = total_map / v_count
        total_recall = total_recall / v_count
    else:
        pass

    return total_auc, total_map, total_recall, v_count, count


def extract_prob(lst):
    #first elements is prob for class 0, extract prob class 0 then 1- prob class 0 = prob class 1
    return [(1-item[0]) for item in lst]

# for i in range(2000):
#     pred_BNB_prob[i] =  extract_prob(pred_BNB_prob[i])

    # print('AUC: ', avg_auc / app_target.shape[0])
    # print('MAP: ', avg_map / app_target.shape[0])
    # print('Recall@5: ', avg_recall / app_target.shape[0])
    # print('Skipped: {} rows, total: {} rows'.format(count, app_target.shape[0]))

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)

# NB Model

In [6]:
from tqdm.notebook import tqdm
from sklearn.naive_bayes import BernoulliNB
from sklearn.multioutput import MultiOutputClassifier

total_user_auc = 0
total_user_map = 0
tota_user_recall = 0
total_valid_rows = 0
total_invalid_rows = 0

for user in tqdm(range(total_user)):
    user_X_train, user_y_train = X_train[user], y_train[user]
    user_X_test, user_y_test = X_test[user], y_test[user]

    nb_clf = MultiOutputClassifier(BernoulliNB()).fit(user_X_train, user_y_train)
    prob = nb_clf.predict_proba(user_X_test)
    
    for apps in range(y_pred.shape[1]):
        prob[apps] = extract_prob(prob[apps])
    y_pred = np.array(prob).T
    # print(y_pred.shape, user_y_test.shape)

    a,m,r,v,c= user_evaluation_metrics(y_pred, user_y_test)
    total_user_auc+=a
    total_user_map+=m
    tota_user_recall+=r
    total_valid_rows+=v
    total_invalid_rows+=c

print('Total user: ', total_user)
print('AUC: ', total_user_auc / total_user)
print('MAP: ', total_user_map / total_user)
print('Recall@5: ', tota_user_recall / total_user)
print('Rows: {} / {}'.format(total_valid_rows, (total_valid_rows + total_invalid_rows)))



C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is 

# MLR Model

In [10]:
from tqdm.notebook import tqdm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

valid_user = 0
total_user_auc = 0
total_user_map = 0
tota_user_recall = 0
total_valid_rows = 0
total_invalid_rows = 0

for user in tqdm(range(total_user)):
    user_X_train, user_y_train = X_train[user], y_train[user]
    user_X_test, user_y_test = X_test[user], y_test[user]

    all0_column_ind = np.argwhere(np.all(user_y_train == 0, axis = 0))
    # print(len(all0_column_ind))
    user_y_train_rm = np.delete(user_y_train, all0_column_ind, axis=1)
    user_y_test_rm = np.delete(user_y_test, all0_column_ind, axis=1)

    try:

        lr_clf = MultiOutputClassifier(LogisticRegression(multi_class = 'ovr', solver = 'saga', n_jobs = -1)).fit(user_X_train, user_y_train_rm)
        prob = lr_clf.predict_proba(user_X_test)
        for apps in range(user_y_train_rm.shape[1]):
            prob[apps] = extract_prob(prob[apps])
        y_pred = np.array(prob).T

    except:
        print(user)
        # pass

    # for apps in range(user_y_train_rm.shape[1]):
    #     prob[apps] = extract_prob(prob[apps])

    # y_pred = np.array(prob).T
    # print(y_pred.shape, user_y_test.shape)

    a,m,r,v,c= user_evaluation_metrics(y_pred, user_y_test_rm)

    total_user_auc+=a
    total_user_map+=m
    tota_user_recall+=r
    total_valid_rows+=v
    total_invalid_rows+=c
    valid_user+=1

    # for apps in range(user_y_train_rm.shape[1]):
    #     prob[apps] = extract_prob(prob[apps])
    # y_pred = np.array(prob).T
    # # print(y_pred.shape, user_y_test.shape)

    # a,m,r,v,c= user_evaluation_metrics(y_pred, user_y_test_rm)

    # total_user_auc+=a
    # total_user_map+=m
    # tota_user_recall+=r
    # total_valid_rows+=v
    # total_invalid_rows+=c

print('Total user: ', valid_user)
print('AUC: ', total_user_auc / valid_user)
print('MAP: ', total_user_map / valid_user)
print('Recall@5: ', tota_user_recall / valid_user)
print('User: {} / {}'.format(valid_user, total_user))
print('Rows: {} / {}'.format(total_valid_rows, (total_valid_rows + total_invalid_rows)))

14
38
41
67
85
90
148
189
393
421
472
592
607
679
695
771
788

Total user:  870
AUC:  0.4354056469924877
MAP:  0.3275366929244181
Recall@5:  0.31708808507400504
User: 870 / 870
Rows: 11043 / 40890
